In [1]:
import numpy as np
import pandas as pd
import re
import collections
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

## 데이터 불러오기

In [3]:
# 데이터 불러오기
data = pd.read_csv('/home/ryu/2023text/final/for_textrank.csv')
data.head()

,title,date,content,url,timeline,year,month
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,2022-07-01 05:39:10,우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를 ...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,2022-07-01 06:20:53,삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다.경쟁사인...,https://n.news.naver.com/mnews/article/214/000...,before,2022,7
3,신한은행 ESG본부 신설 하반기 정기 인사,2022-07-01 15:25:08,AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 자...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,2022-07-01 11:14:20,영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서 ...,https://n.news.naver.com/mnews/article/003/001...,before,2022,7


In [ ]:
# Null값 확인
data[data['content'].isnull()]

In [140]:
data.dropna(subset=['content'], axis=0, inplace=True)

In [143]:
data['content'].isnull().sum()

0

## 문장 분리

In [144]:
# 문장 분리 함수
def split_sent(content):
    after = re.sub('[^a-zA-Zㄱ-ㅎ가-힣]\.', 'xx', content).split('.')
    return after

In [145]:
data['sentences'] = data['content'].apply(lambda x: split_sent(x))

In [147]:
data['sentences'][40008]

['어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소재 다양한 교육 체험 프로그램 마련마산로봇랜드 전경',
 ' 경남도 제공경남도는 5월 가정의 달을 맞아 마산로봇랜드 테마파크에 콘텐츠를 보강하고 다채로운 볼거리와 체험프로그램을 운영한다고 30일 밝혔다',
 '로봇랜드는 다음달 부터 중앙 드림광장에서 풍선과 비눗방울을 활용한 마술 벌룬 버블쇼를 진행한다',
 ' 주말에는 xx4m의 강철 로봇이 거리를 돌아다니며 다양한 퍼포먼스를 펼치는 타이탄 로봇쇼 를 감상할 수 있다',
 ' 특히 어린이날 연휴 동안 마칭밴드가 로봇마차와 함께 행진하는 퍼레이드를 벌이고 로봇놀이터에서 작은 고철들을 활용해 본인만의 개성 있는 로봇을 만드는 정크야 놀자 직접 로봇 가면을 제작하고 채색하는 로봇가면 만들기 노래에 맞춰 춤추는 댄스로봇의 공연 드론을 조종하여 장애물을 피하는 드론조종 체험 등 아이들과 온 가족이 함께 즐기는 콘텐츠를 운영한다',
 '또 로봇가면 만들기 는 본인만의 가면을 만들어 착용하고 사회자의 안내에 따라 무궁화 꽃이 피었습니다 등 간단한 게임 이벤트를 진행하고 식음로봇인 솜사탕 로봇 과 라면 끓이는 로봇 을 테마파크 내에 적절히 배치해 방문객에게 볼거리와 먹거리를 제공한다',
 '이와 함께 로봇랜드 공공관 중 하나인 미래로봇관에 11대의 로봇을 도입해 미래 로봇세상을 표현할 예정이다',
 '인공지능 화가 로봇 4대를 도입해 방문객 초상화를 그려 제공하는 서비스도 선보인다',
 '로봇을 소재로 한 다양한 교육 체험프로그램도 마련한다',
 '로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 쾌속열차 스카이타워 등의 과학원리를 학습하고 관련 교구를 만드는 로봇랜드 어트랙션 연계 로봇 제작 사물지능융합기술을 활용한 스마트홈 제작 홀로그램 기반 K POP 영상 제작 메타버스 공간을 만드는 인공지능 기술 체험 등을 할 수 있다',
 '로봇체험관을 활용한 해설이 있는 에듀투어 에서는 유 초 중 고 학생을 대상

### 분리 후 공백처리

In [148]:
def clean(sent):
    new_text = re.sub(r"^\s+", '', sent) # space from start 제거
    new_text = re.sub(r'\s+$', '', new_text) # space from the end 제거
    new_text = re.sub(r'\s{2,}', ' ', new_text) # 다중 공백 -> 하나의 공백으로

    return new_text


def clean_series(series):
    for i in range(len(series)):
        series[i] = clean(series[i])
    
    return series

In [149]:
data['sentences'] = data['sentences'].apply(lambda x: clean_series(x))

In [150]:
data['sentences'][40008]

['어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소재 다양한 교육 체험 프로그램 마련마산로봇랜드 전경',
 '경남도 제공경남도는 5월 가정의 달을 맞아 마산로봇랜드 테마파크에 콘텐츠를 보강하고 다채로운 볼거리와 체험프로그램을 운영한다고 30일 밝혔다',
 '로봇랜드는 다음달 부터 중앙 드림광장에서 풍선과 비눗방울을 활용한 마술 벌룬 버블쇼를 진행한다',
 '주말에는 xx4m의 강철 로봇이 거리를 돌아다니며 다양한 퍼포먼스를 펼치는 타이탄 로봇쇼 를 감상할 수 있다',
 '특히 어린이날 연휴 동안 마칭밴드가 로봇마차와 함께 행진하는 퍼레이드를 벌이고 로봇놀이터에서 작은 고철들을 활용해 본인만의 개성 있는 로봇을 만드는 정크야 놀자 직접 로봇 가면을 제작하고 채색하는 로봇가면 만들기 노래에 맞춰 춤추는 댄스로봇의 공연 드론을 조종하여 장애물을 피하는 드론조종 체험 등 아이들과 온 가족이 함께 즐기는 콘텐츠를 운영한다',
 '또 로봇가면 만들기 는 본인만의 가면을 만들어 착용하고 사회자의 안내에 따라 무궁화 꽃이 피었습니다 등 간단한 게임 이벤트를 진행하고 식음로봇인 솜사탕 로봇 과 라면 끓이는 로봇 을 테마파크 내에 적절히 배치해 방문객에게 볼거리와 먹거리를 제공한다',
 '이와 함께 로봇랜드 공공관 중 하나인 미래로봇관에 11대의 로봇을 도입해 미래 로봇세상을 표현할 예정이다',
 '인공지능 화가 로봇 4대를 도입해 방문객 초상화를 그려 제공하는 서비스도 선보인다',
 '로봇을 소재로 한 다양한 교육 체험프로그램도 마련한다',
 '로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 쾌속열차 스카이타워 등의 과학원리를 학습하고 관련 교구를 만드는 로봇랜드 어트랙션 연계 로봇 제작 사물지능융합기술을 활용한 스마트홈 제작 홀로그램 기반 K POP 영상 제작 메타버스 공간을 만드는 인공지능 기술 체험 등을 할 수 있다',
 '로봇체험관을 활용한 해설이 있는 에듀투어 에서는 유 초 중 고 학생을 대상으로 

## 형태소 분석 및 불용어 추출

In [4]:
from konlpy.tag import Mecab 
mecab = Mecab()
mecab_path = '/home/ryu/mecab-ko-dic-2.1.1-20180720'

In [5]:
stop = pd.read_csv('/home/ryu/2023text/final/ai_stopwords.txt', header=None)
# stop.head()

In [6]:
stopwords = [i for i in stop[0]]

def mecab_tokenize(s):
    nouns = mecab.nouns(s)
    # return [noun for noun in nouns]
    return [noun for noun in nouns if noun not in stopwords]

## TextRank 요약

In [7]:
import textrank
from textrank.textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(tokenize=mecab_tokenize, min_sim=0.5)

In [8]:
import textrank
from textrank.textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(tokenize=mecab_tokenize, min_sim=0.5)

def text_summarize(content):
    try:
        keys = []

        if len(content) <= 2:
            top = len(content)
        else:
            top = int((len(content))*0.5)
        
        keysents = summarizer.summarize(content, topk=top)
        for k in keysents:
            keys.append(k[2])
    except:
        print('error occurred')
        pass
    
    return keys


In [ ]:
data['summarization'] = data['sentences'].progress_apply(lambda x: text_summarize(x))

In [10]:
print('전체 데이터:', len(data))
print('요약에 성공한 데이터:', len(data['summarization']))

전체 데이터: 40003
요약에 성공한 데이터: 40003


In [12]:
data.head()

,title,date,content,url,timeline,year,month,sentences,summarization
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,2022-07-01 05:39:10,우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를 ...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7,[우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를...,[폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 최...
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,2022-07-01 06:20:53,삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다.경쟁사인...,https://n.news.naver.com/mnews/article/214/000...,before,2022,7,"[삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다, 경쟁...","[이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다, 삼..."
3,신한은행 ESG본부 신설 하반기 정기 인사,2022-07-01 15:25:08,AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 자...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7,[AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 ...,[신한은행에 따르면 이번 인사는 본점 영업점 장기근속 직원의 순환 근무를 통해 각종...
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,2022-07-01 11:14:20,영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서 ...,https://n.news.naver.com/mnews/article/003/001...,before,2022,7,[영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서...,[영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서...


In [15]:
data.drop(['date', 'content', 'url'], axis=1, inplace=True)

In [17]:
data.head()

,title,timeline,year,month,sentences,summarization
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,before,2022,7,[우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를...,[폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 최...
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,before,2022,7,"[삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다, 경쟁...","[이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다, 삼..."
3,신한은행 ESG본부 신설 하반기 정기 인사,before,2022,7,[AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 ...,[신한은행에 따르면 이번 인사는 본점 영업점 장기근속 직원의 순환 근무를 통해 각종...
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,before,2022,7,[영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서...,[영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서...


In [18]:
# 데이터 저장
data.to_csv('/home/ryu/2023text/final/upto_summarization_0603.csv', index=False, encoding='utf-8')

## 데이터 후처리
(길이가 200을 넘는 summary의 경우 제거)

In [7]:
summ = pd.read_csv('/home/ryu/2023text/final/upto_summarization_0603.csv')
summ.head()


,title,timeline,year,month,sentences,summarization
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 ...,['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 ...
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,before,2022,7,['우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개...,['폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 ...
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,before,2022,7,"['삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다', ...","['이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다',..."
3,신한은행 ESG본부 신설 하반기 정기 인사,before,2022,7,['AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일...,['신한은행에 따르면 이번 인사는 본점 영업점 장기근속 직원의 순환 근무를 통해 각...
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,before,2022,7,['영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에...,['영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에...


In [8]:
summ = summ.drop_duplicates(subset='summarization', keep='first', ignore_index=True)

In [10]:
summ = summ[summ['summarization']!='[]']

In [13]:
summ[summ.duplicated(subset='summarization')]

,title,timeline,year,month,sentences,summarization


In [14]:
len(summ)

39726

In [15]:
# 댓글은 다시 리스트형으로 변환 (데이터 가져오면서 리스트를 문자열로 인식)
summ['summarization']=summ['summarization'].str.replace('[','').str.replace(']','').str.replace('\'','') #nouns 칼럼에 있는 []\문자 제거
summ['summarization']=summ['summarization'].apply(lambda x:x.split(',')) # ,를 기준으로 분리 후 리스트에 담기
summ['summarization']=summ['summarization'].apply(lambda x:[i.strip() for i in x]) # 리스트 내 문자 양쪽 공백제거

/tmp/ipykernel_29709/4086137079.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  summ['summarization']=summ['summarization'].str.replace('[','').str.replace(']','').str.replace('\'','') #nouns 칼럼에 있는 []\문자 제거


In [21]:
summ = summ[summ['summarization'].apply(len)<200]

In [22]:
summ.head()

,title,timeline,year,month,sentences,summarization
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 ...,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,before,2022,7,['우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개...,[폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 최...
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,before,2022,7,"['삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다', ...","[이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다, 삼..."
3,신한은행 ESG본부 신설 하반기 정기 인사,before,2022,7,['AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일...,[신한은행에 따르면 이번 인사는 본점 영업점 장기근속 직원의 순환 근무를 통해 각종...
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,before,2022,7,['영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에...,[영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서...


In [23]:
# 데이터 저장
summ.to_csv('/home/ryu/2023text/final/textrank_results_0605.csv', index=False, encoding='utf-8')

In [2]:
df = pd.read_csv('/home/ryu/2023text/final/textrank_results_0605.csv')
len(df)

39719

In [4]:
len(df[df['timeline']=='before'])

18775

In [5]:
len(df[df['timeline']=='after'])

20944